# GOTM at Southern Ocean Flux Station

This notebook runs [GOTM](https://gotm.net/) with initial conditions and surface forcing according to the LES data of [Large et al., 2019](https://doi.org/10.1175/JPO-D-18-0066.1).

In [ ]:
import sys
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *

In [ ]:
casename = 'AprS'
if casename == 'AprS':
    starttime = '2010-04-29 00:00:00'
    stoptime   = '2010-05-01 04:30:00'
    depth = 350.0
elif casename == "AprN":
    starttime = "2010-04-29 00:00:00"
    stoptime  = "2010-05-01 02:00:00"
    depth = 350.0
elif casename== "JunS":
    starttime = "2010-06-07 06:45:00"
    stoptime  = "2010-06-08 23:45:00"
    depth = 500.0
elif casename == "JunN":
    starttime = "2010-06-07 06:45:00"
    stoptime  = "2010-06-08 21:15:00"
    depth = 500.0
else:
    raise ValueError('Case {:s} not supported. Stop.'.format(casename))

## Create a model
Create a model with environment file `gotmtool/.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [ ]:
m = Model(name='SOFS-{:s}'.format(casename), environ=os.path.join(os.pardir, 'gotmtool', '.gotm_env.yaml'))

In [ ]:
os.getcwd()

Take a look at what are defined in the environment file.

In [ ]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

## Build the model

In [ ]:
%%time
m.build()

## Configuration
Initialize the GOTM configuration

In [ ]:
cfg = m.init_config()

Update the configuration

In [ ]:
# setup
title = 'SOFS - {:s}'.format(casename)
nlev = int(depth)
cfg['title'] = title
cfg['location']['name'] = 'SOFS'
cfg['location']['latitude'] = -47.0
cfg['location']['longitude'] = 141.9
cfg['location']['depth'] = depth
cfg['time']['start'] = starttime
cfg['time']['stop']  = stoptime
cfg['time']['dt']    = 600.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 3
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = os.path.join(os.getcwd(), 'data_{:s}'.format(casename))
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = os.path.join(datadir, 't_prof.dat')
cfg['salinity']['method'] = 'constant'
cfg['salinity']['constant_value'] = 35.0
cfg['surface']['fluxes']['heat']['method'] = 'file'
cfg['surface']['fluxes']['heat']['file'] = os.path.join(datadir, 'heat_flux.dat')
cfg['surface']['fluxes']['tx']['method'] = 'file'
cfg['surface']['fluxes']['tx']['file'] = os.path.join(datadir, 'momentum_flux.dat')
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['ty']['method'] = 'file'
cfg['surface']['fluxes']['ty']['file'] = os.path.join(datadir, 'momentum_flux.dat')
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['swr']['method'] = 'constant'
cfg['surface']['swr']['constant_value'] = 0.0
cfg['surface']['precip']['method'] = 'constant'
cfg['surface']['precip']['constant_value'] = 0.0
cfg['velocities']['u']['method'] = 'file'
cfg['velocities']['u']['file'] = os.path.join(datadir, 'vel_prof.dat')
cfg['velocities']['u']['column'] = 1
cfg['velocities']['v']['method'] = 'file'
cfg['velocities']['v']['file'] = os.path.join(datadir, 'vel_prof.dat')
cfg['velocities']['v']['column'] = 2
cfg['waves']['stokes_drift']['us0']['method'] = 'file'
cfg['waves']['stokes_drift']['us0']['file'] = os.path.join(datadir, 'us0.dat')
cfg['waves']['stokes_drift']['us0']['column'] = 1
cfg['waves']['stokes_drift']['vs0']['method'] = 'file'
cfg['waves']['stokes_drift']['vs0']['file'] = os.path.join(datadir, 'us0.dat')
cfg['waves']['stokes_drift']['vs0']['column'] = 2
cfg['waves']['stokes_drift']['us']['method'] = 'file'
cfg['waves']['stokes_drift']['us']['file'] = os.path.join(datadir, 'us_prof.dat')
cfg['waves']['stokes_drift']['us']['column'] = 1
cfg['waves']['stokes_drift']['vs']['method'] = 'file'
cfg['waves']['stokes_drift']['vs']['file'] = os.path.join(datadir, 'us_prof.dat')
cfg['waves']['stokes_drift']['vs']['column'] = 2

# EOS -- use linear
cfg['equation_of_state']['method'] = 'linear_custom'
cfg['equation_of_state']['linear']['T0'] = 12.0
cfg['equation_of_state']['linear']['S0'] = 35.0
cfg['equation_of_state']['linear']['alpha'] = 1.668e-4
cfg['equation_of_state']['linear']['beta'] = 7.81e-4
cfg['equation_of_state']['linear']['cp'] = 4200
cfg['equation_of_state']['rho0'] = 1000

Now run the case with [CVMix](http://cvmix.github.io).
- KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
- StokesMOST ([Large et al., 2021](https://doi.org/10.1175/JPO-D-20-0308.1))

In [ ]:
cfgs = []
labels = []

In [ ]:
cfg['turbulence']['turb_method'] = 'cvmix'
cfg['cvmix']['surface_layer']['langmuir_method'] = 'lf17'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-LF17')
cfg['cvmix']['surface_layer']['langmuir_method'] = 'none'
cfg['cvmix']['surface_layer']['use_Stokes_MOST'] = True
cfgs.append(copy.deepcopy(cfg))
labels.append('StokesMOST')

## Run the model

In [ ]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

## Results

Load the data into an `xarray.Dataset`.

In [ ]:
data1 = sims[0].load_data()
data2 = sims[1].load_data()

In [ ]:
fig, axarr = plt.subplots(2, 1, sharex='col')
fig.set_size_inches([8,6])
data1.u.plot(ax=axarr[0])
data2.u.plot(ax=axarr[1])